# Scrape Appointees from Orders in Council

## Get Search Page

In [1]:
import mechanicalsoup
import re

In [2]:
browser = mechanicalsoup.StatefulBrowser()

In [3]:
url = "https://orders-in-council.canada.ca/index.php?lang=en"

In [4]:
browser.open(url)

<Response [200]>

## Enter Form Search Criteria

In [5]:
browser.select_form()

In [6]:
browser["fromDate"] = "1993-01-01"

In [7]:
browser.form.print_summary()

<input class="form-control inputText" id="pcNumber" name="pcNumber" placeholder="yyyy-nnnn" type="text"/>
<input class="form-control inputText" name="fromDate" placeholder="yyyy-mm-dd" type="text" value="1993-01-01"/>
<input class="form-control inputText" name="toDate" placeholder="yyyy-mm-dd" type="text"/>
<input class="form-control inputText" name="keywords" type="text"/>
<input class="form-control inputText" name="department" type="text"/>
<input class="form-control inputText" name="act" type="text"/>
<input class="form-control inputText" name="chapterNumber" type="text"/>
<input class="form-control inputText" name="chapterYear" type="text"/>
<input class="form-control inputText" name="billNumber" type="text"/>
<input name="foa" type="radio" value="1"/>
<input name="foa" type="radio" value="0"/>
<input checked="" name="foa" type="radio" value="na"/>
<input class="btn btn-primary" id="btnSearch" name="searchList" type="submit" value="Search/ List"/>
<input class="btn btn-default" nam

In [8]:
#browser.launch_browser()

## Submit Form and Retrieve Results Page

In [9]:
response = browser.submit_selected()

In [10]:
print(response.text)

﻿﻿<!DOCTYPE html>
<!--[if lt IE 9]><html class="no-js lt-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if gt IE 8]><!-->
<html xmlns="http://www.w3.org/1999/xhtml" class="no-js" lang="en" dir="ltr">
<!--<![endif]-->
<head>
    <meta charset="utf-8" />
    <!-- Web Experience Toolkit (WET) / Boite a outils de lexperience Web (BOEW)
        wet-boew.github.io/wet-boew/License-en.html / wet-boew.github.io/wet-boew/Licence-fr.html -->
    <title>Orders In Council - Search</title>
    <meta content="width=device-width,initial-scale=1" name="viewport" />
    <link rel="shortcut icon" href="/wet-boew/theme-gcwu-fegc/images/favicon.ico" />
    <meta name="description" content="Orders In Council - Search" />
    <meta name="dcterms.creator" content="Government of Canada" />
    <meta name="dcterms.title" content="Orders In Council - Search" />
    <meta name="dcterms.subject" title="scheme" content="Politics" />
    <meta name="dcterms.language" title="ISO639-2" content="en" />
    <meta name="vie

In [11]:
browser.list_links()

Links in the current page:
     <a class="pagebutton" href="results.php?pageNum=1&amp;lang=en"><span class="currentpage btn btn-default">1</span></a>
     <a class="pagebutton" href="results.php?pageNum=2&amp;lang=en"><span class="btn btn-default">2</span></a>
     <a class="pagebutton" href="results.php?pageNum=3&amp;lang=en"><span class="btn btn-default">3</span></a>
     <a class="pagebutton" href="results.php?pageNum=4&amp;lang=en"><span class="btn btn-default">4</span></a>
     <a class="pagebutton" href="results.php?pageNum=5&amp;lang=en"><span class="btn btn-default">5</span></a>
     <a class="pagebutton" href="results.php?pageNum=6&amp;lang=en"><span class="btn btn-default">6</span></a>
     <a class="pagebutton" href="results.php?pageNum=10561&amp;lang=en"><span class="btn btn-default first">10561</span></a>
     <a href="attachment.php?attach=40168&amp;lang=en">Attachment</a>
     <a href="attachment.php?attach=40165&amp;lang=en">Attachment</a>
     <a href="attachment.php?a

## Find Highest Page Number Returned

In [12]:
highest_page_number = browser.page.find("span", "btn btn-default first").string
type(highest_page_number)
highest_page_number

'10561'

## Follow Link to First Attachment

In [13]:
browser.links("attachment")

[<a href="attachment.php?attach=40168&amp;lang=en">Attachment</a>,
 <a href="attachment.php?attach=40165&amp;lang=en">Attachment</a>,
 <a href="attachment.php?attach=40164&amp;lang=en">Attachment</a>,
 <a href="attachment.php?attach=40163&amp;lang=en">Attachment</a>,
 <a href="attachment.php?attach=40162&amp;lang=en">Attachment</a>]

In [14]:
#browser.links("attachment")

for i in browser.links("attachment"):
    browser.follow_link(i)
    print(browser.url)
    soup = browser.page
    soup.select("p:-soup-contains(Minister)")

https://orders-in-council.canada.ca/attachment.php?attach=40168&lang=en
https://orders-in-council.canada.ca/attachment.php?attach=40165&lang=en
https://orders-in-council.canada.ca/attachment.php?attach=40164&lang=en
https://orders-in-council.canada.ca/attachment.php?attach=40163&lang=en
https://orders-in-council.canada.ca/attachment.php?attach=40162&lang=en


In [15]:
soup.select("p:-soup-contains(Minister)")

[<p style="text-indent:1.00in;text-align:left;margin-top:0.00in;margin-bottom:0.00in;">
 Her Excellency the Governor General in Council, on the recommendation of the Minister of Canadian Heritage, pursuant to section 18 of the <em>Cultural Property Export and Import Act</em>, reappoints Laurie Dalton of Wolfville, Nova Scotia, as a member of the Canadian Cultural Property Export Review Board, to hold office during pleasure for a term of three years.</p>]